|<h2>Book:</h2>|<h1><a href="https://open.substack.com/pub/mikexcohen/p/llm-breakdown-16-tokenization-words" target="_blank">50 ML projects to understand LLMs</a></h1>|
|-|:-:|
|<h2>Project:</h2>|<h1><b>[34] QKV activation characteristics</b></h1>|
|<h2>Author:<h2>|<h1>Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h1>|

<br>

<i>Using the code without reading the book may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import requests

import torch
from transformers import AutoModelForCausalLM, GPT2Tokenizer
import torch.nn.functional as F

In [ ]:
### matplotlib adjustments (commented lines are for dark mode)

# svg plots (higher-res)
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

plt.rcParams.update({
    # 'figure.facecolor': '#282a2c',
    # 'figure.edgecolor': '#282a2c',
    # 'axes.facecolor':   '#282a2c',
    # 'axes.edgecolor':   '#DDE2F4',
    # 'axes.labelcolor':  '#DDE2F4',
    # 'xtick.color':      '#DDE2F4',
    # 'ytick.color':      '#DDE2F4',
    # 'text.color':       '#DDE2F4',
    'axes.spines.right': False,
    'axes.spines.top':   False,
    'axes.titleweight': 'bold',
    'axes.labelweight': 'bold',
    'savefig.dpi':300
})

# **Part 1: Model, tokens, QKV activations**

In [ ]:
# load GPT2 model and tokenizer
model = AutoModelForCausalLM.from_pretrained('gpt2-large')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

model.eval()

In [ ]:
# variable for the number of embedding dimensions
n_emb = model.config.n_embd

# and the number of layers
n_layers = model.config.n_layer

In [ ]:
# GreatGatsby
url = 'https://www.gutenberg.org/cache/epub/64317/pg64317.txt'
text = requests.get(url).text
tokens = np.array( tokenizer.encode(text) )

target_word = ' like'

target_idx = np.where(tokens==tokenizer.encode(target_word))[0]
print(f'{len(target_idx)} instances of "{target_word}"')

In [ ]:
# size of pre-target context (tokens)
contextwin = 50

# remove any target indices that are <contextwin
print(f'Target count before filtering: {len(target_idx)}')
target_idx = target_idx[target_idx >= contextwin]

print(f'Target count after filtering: {len(target_idx)}')

In [ ]:
# create a batch
batch = torch.zeros((len(target_idx),contextwin+1),dtype=torch.long)

for i in range(len(target_idx)):
  toks = tokens[target_idx[i]-contextwin:target_idx[i]+1]
  batch[i,:] = torch.tensor(toks)

batch.shape

In [ ]:
# note the identical final token
batch

In [ ]:
# a few examples
for i in range(5):
  print(tokenizer.decode(batch[i,-10:]))

In [ ]:
# Define a hook function to store QKV vectors
activations = {}

def implant_hook(layer_number):
  def hook(module,input,output):
    activations[f'attn_{layer_number}_qkv'] = output.detach()
  return hook


# surgery ;)
handles = []
for i in range(n_layers):
  h = model.transformer.h[i].attn.c_attn.register_forward_hook(implant_hook(i))
  handles.append(h)

In [ ]:
# push through the model
with torch.no_grad(): model(batch)

for k,i in activations.items():
  print(f'{k:>12} has shape {list(i.shape)}')

In [ ]:
# concatenated activations from one layer
layeri = 6

wide_acts = activations[f'attn_{layeri}_qkv']

plt.figure(figsize=(10,3))
plt.imshow(wide_acts[:,-1,:],aspect='auto',vmin=-1,vmax=1,cmap='plasma')
plt.axvline(n_emb,linestyle='--',color='w')
plt.axvline(2*n_emb,linestyle='--',color='w')
plt.colorbar(pad=.01)

plt.gca().set(xticks=[],ylabel='Batch sequences',
              xlabel='Queries dimensions         |           Keys dimensions             |           Values dimensions')


plt.tight_layout()
plt.savefig('ch6_proj34_part1.png')
plt.show()

# **Part 2: Activation characteristics**

In [ ]:
# just one layer for now
layeri = 6

# split into separate matrices
Q,K,V = torch.split(activations[f'attn_{layeri}_qkv'],n_emb,dim=-1)
Q.shape,K.shape,V.shape

In [ ]:
# target: means of means and variances
qMeans_t = torch.tensor([ torch.mean(Q[:,-1,:],dim=-1).mean(),
                          torch.mean(abs(Q[:,-1,:]),dim=-1).mean(),
                          torch.var(Q[:,-1,:],dim=-1).mean() ])

# non-target: means of means and variances
qMeans_n = torch.tensor([ torch.mean(Q[:,-2,:],dim=-1).mean(),
                          torch.mean(abs(Q[:,-2,:]),dim=-1).mean(),
                          torch.var(Q[:,-2,:],dim=-1).mean() ])


plt.figure(figsize=(8,3))
plt.bar(np.arange(3)-.2,qMeans_t,width=.5,edgecolor='k',label='Target')
plt.bar(np.arange(3)+.2,qMeans_n,width=.5,edgecolor='k',label='Non-target')
plt.legend()
plt.gca().set(xticks=range(3),xticklabels=['Arithmetic mean','L1 mean','Variance'],ylabel='Descriptive values')

plt.tight_layout()
plt.savefig('ch6_proj34_part2a.png')
plt.show()

In [ ]:
edges = torch.linspace(-7,7,234)

ytQ,_ = torch.histogram(Q[:,-1,:].flatten(),edges,density=True)
ynQ,_ = torch.histogram(Q[:,-2,:].flatten(),edges,density=True)
ytK,_ = torch.histogram(K[:,-1,:].flatten(),edges,density=True)
ynK,_ = torch.histogram(K[:,-2,:].flatten(),edges,density=True)
ytV,_ = torch.histogram(V[:,-1,:].flatten(),edges,density=True)
ynV,_ = torch.histogram(V[:,-2,:].flatten(),edges,density=True)

plt.figure(figsize=(10,3))
plt.plot(edges[:-1],ytQ,'k',label='Q target')
plt.plot(edges[:-1],ynQ,'k--',label='Q nontarget')
plt.plot(edges[:-1],ytK,'g',label='K target')
plt.plot(edges[:-1],ynK,'g--',label='K nontarget')
plt.plot(edges[:-1],ytV,'r',label='V target')
plt.plot(edges[:-1],ynV,'r--',label='V nontarget')

plt.legend()
plt.gca().set(xlim=edges[[0,-1]],xlabel='Activation value',ylabel='Density',
              title=f'Attention matrices distributions from layer {layeri}')

plt.tight_layout()
plt.savefig('ch6_proj34_part2b.png')
plt.show()

# **Part 3: Laminar trends in activation descriptives**

In [ ]:
# helper function
def meansAndVar(X):

  # target
  t1 = torch.mean(X[:,-1,:],dim=-1).mean()      # arithmetic mean
  t2 = torch.mean(abs(X[:,-1,:]),dim=-1).mean() # L1 mean
  t3 = torch.var(X[:,-1,:],dim=-1).mean()       # variance

  # non-target
  n1 = torch.mean(X[:,-2,:],dim=-1).mean()      # arithmetic mean
  n2 = torch.mean(abs(X[:,-2,:]),dim=-1).mean() # L1 mean
  n3 = torch.var(X[:,-2,:],dim=-1).mean()       # variance

  return torch.tensor([[t1,t2,t3],[n1,n2,n3]])

# test
meansAndVar(Q)

In [ ]:
# initialize: layers X matrix X feature X category
descriptives = torch.zeros((n_layers,3,3,2))

for layeri in range(n_layers):

  # split into separate matrices
  Q,K,V = torch.split(activations[f'attn_{layeri}_qkv'],n_emb,dim=-1)

  # Q: get the descriptives
  T,N = meansAndVar(Q)
  descriptives[layeri,0,:,0] = T
  descriptives[layeri,0,:,1] = N

  # K: get the descriptives
  T,N = meansAndVar(K)
  descriptives[layeri,1,:,0] = T
  descriptives[layeri,1,:,1] = N

  # V: get the descriptives
  T,N = meansAndVar(V)
  descriptives[layeri,2,:,0] = T
  descriptives[layeri,2,:,1] = N


descriptives.shape

In [ ]:
_,axs = plt.subplots(1,3,figsize=(12,3))

feature = ['Arithmetic mean','L1 mean','Variance']

for i in range(len(axs)):
  axs[i].plot(descriptives[:,0,i,:].mean(dim=-1),'gs-',markerfacecolor=[.7,.9,.7,.7],label='Q')
  axs[i].plot(descriptives[:,1,i,:].mean(dim=-1),'ro-',markerfacecolor=[.9,.7,.7,.7],label='K')
  axs[i].plot(descriptives[:,2,i,:].mean(dim=-1),'b^-',markerfacecolor=[.7,.7,.9,.7],label='V')
  axs[i].legend()
  axs[i].set(xlabel='Transformer layer',xticks=range(0,n_layers,4),title=f'{feature[i]}')

plt.tight_layout()
plt.savefig('ch6_proj34_part3a.png')
plt.show()

In [ ]:
_,axs = plt.subplots(1,3,figsize=(12,3))

for i in range(len(axs)):
  axs[i].axhline(0,linestyle='--',linewidth=.5,color=[.7,.7,.7])
  axs[i].plot(descriptives[:,0,i,:].diff(dim=-1),'gs-',markerfacecolor=[.7,.9,.7,.7],label='Q')
  axs[i].plot(descriptives[:,1,i,:].diff(dim=-1),'ro-',markerfacecolor=[.9,.7,.7,.7],label='K')
  axs[i].plot(descriptives[:,2,i,:].diff(dim=-1),'b^-',markerfacecolor=[.7,.7,.9,.7],label='V')
  axs[i].legend()
  axs[i].set(xlabel='Transformer layer',xticks=range(0,n_layers,4),title=f'{feature[i]}')

plt.tight_layout()
plt.savefig('ch6_proj34_part3b.png')
plt.show()

# **Part 4: Laminar trends in activation distributions**

In [ ]:
# helper function
edges = torch.linspace(-4,4,101)

def calculateHists(X):
  yt,_ = torch.histogram(X[:,-1,:].flatten(),edges,density=True)
  yn,_ = torch.histogram(X[:,-2,:].flatten(),edges,density=True)
  return yt,yn

In [ ]:
# initialize: layers X matrix X bins X category
histograms = torch.zeros((n_layers,3,len(edges)-1,2))

for layeri in range(n_layers):

  # split into separate matrices
  Q,K,V = torch.split(activations[f'attn_{layeri}_qkv'],n_emb,dim=-1)

  # get the histograms
  histograms[layeri,0,:,0],histograms[layeri,0,:,1] = calculateHists(Q)
  histograms[layeri,1,:,0],histograms[layeri,1,:,1] = calculateHists(K)
  histograms[layeri,2,:,0],histograms[layeri,2,:,1] = calculateHists(V)

histograms.shape

In [ ]:
fig,axs = plt.subplots(2,3,figsize=(12,6))

for i in range(3):
  h = axs[0,i].imshow(histograms[:,i,:,0],vmin=0,vmax=.4,aspect='auto',cmap='plasma',
                  extent=[edges[0],edges[-1],0,n_layers],origin='lower')
  axs[0,i].set(xticklabels=[],ylabel='Layer',yticks=range(0,n_layers,5),title=f"Target {'QKV'[i]}")
  fig.colorbar(h,ax=axs[0,i],pad=.03,orientation='horizontal')

  axs[1,i].imshow(histograms[:,i,:,1],vmin=0,vmax=.4,aspect='auto',cmap='plasma',
                  extent=[edges[0],edges[-1],0,n_layers],origin='lower')
  axs[1,i].set(xlabel='Activation value',ylabel='Layer',yticks=range(0,n_layers,5),title=f"Non-target {'QKV'[i]}")


plt.tight_layout()
plt.savefig('ch6_proj34_part4.png')
plt.show()

# **Part 5: Extracting and characterizing heads**

In [ ]:
n_heads = model.config.n_head
head_dim = n_emb // n_heads

In [ ]:
# extract Q
Q,K,V = torch.split(activations['attn_6_qkv'],n_emb,dim=-1)

# and reshape to have a "head" dimension
Qh = Q.view(batch.shape[0],batch.shape[1],n_heads,head_dim)

print(f'Q is size {list(Q.shape)} and Qh is size {list(Qh.shape)}')

In [ ]:
# visualize
_,axs = plt.subplots(4,5,figsize=(12,6))

for i,ax in enumerate(axs.flatten()):
  ax.pcolor(Qh[0,:,i,:].T,cmap='plasma',vmin=-2,vmax=2)
  ax.text(contextwin-.5,head_dim-1,f'Qh{i}',fontsize=12,fontweight='bold',color='k',ha='right',va='top')
  ax.text(contextwin,head_dim-2,f'Qh{i}',fontsize=12,fontweight='bold',color='w',ha='right',va='top')
  ax.set(xticks=[],yticks=[])

# finalize
axs[3,0].set(ylabel='Head dim',xlabel='Token position')

plt.tight_layout()
plt.savefig('ch6_proj34_part5a.png')
plt.show()

In [ ]:
head_stdevs = np.zeros((n_layers,3))

for layeri in range(n_layers):

  # split into Q, K, and V
  Q,K,V = torch.split(activations[f'attn_{layeri}_qkv'],n_emb,dim=-1)

  # reshape to have a heads dimension
  Qh = Q.view(batch.shape[0],batch.shape[1],n_heads,head_dim)
  Kh = K.view(batch.shape[0],batch.shape[1],n_heads,head_dim)
  Vh = V.view(batch.shape[0],batch.shape[1],n_heads,head_dim)

  # then get the standard deviation along the tokens
  head_stdevs[layeri,0] = Qh.std(dim=1).mean()
  head_stdevs[layeri,1] = Kh.std(dim=1).mean()
  head_stdevs[layeri,2] = Vh.std(dim=1).mean()


# and visualize
plt.figure(figsize=(10,3))

plt.plot(head_stdevs[:,0],'gs-',markersize=9,markerfacecolor=[.7,.9,.7],label='Q')
plt.plot(head_stdevs[:,1],'ro-',markersize=9,markerfacecolor=[.9,.7,.7],label='K')
plt.plot(head_stdevs[:,2],'b^-',markersize=9,markerfacecolor=[.7,.7,.9],label='V')

plt.legend()
plt.gca().set(xlabel='Transformer layer',ylabel='Average standard deviation')

plt.tight_layout()
plt.savefig('ch6_proj34_part5b.png')
plt.show()